Final


In [ ]:
# import libraries
import os
os.system('pip install swifter')
os.system('pip install pyunpack')
os.system('pip install patool')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.linear_model import SGDRegressor
import lightgbm as lgb
from pyunpack import Archive
import swifter
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, vstack,hstack
import re
import array
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from sklearn.feature_extraction.text import _make_int_array, CountVectorizer
from scipy import sparse as sp
import multiprocessing as mp
import pickle
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MaxAbsScaler
import scipy.sparse
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")


# DL Libs..
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input , Dropout, Flatten,concatenate,LSTM
from tensorflow.keras.layers import Embedding
import tensorflow.keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

####################################################################################################
def func_extract_zip(in_path,out_path):

  #ref - https://www.kaggle.com/general/129520
  if not os.path.exists(out_path):
      os.makedirs(out_path)
  Archive(in_path).extractall(out_path)


def func_load_data():

  df_train = pd.read_csv('train.tsv',sep = '\t')
  df_test = pd.read_csv('test_stg2.tsv',sep = '\t')
  return df_train,df_test


def imputation(data):

  """
  Fill the missing values
  """
  data.category_name.fillna(value = "other/other/other", inplace = True)
  data.brand_name.fillna(value = "unknown", inplace = True)
  data.item_description.fillna(value = "no_description", inplace = True)
  return data
# pre-process shipping

def pre_shipping(df):
  """
  This function takes df as input and converts shipping column into csr matrix and returns that
  i/p= Whole Dataframe
  o/p= Shipping coloumn in csr matrix format
  o/p shape = (?,1)
  """
  shipping = np.array(df['shipping'])
  shipping = shipping.reshape(-1,1)
  shipping = csr_matrix(shipping)

  return shipping


def pre_item_condition_id(df,vectorizer=None,train=True):

  """
  This function is to pre-process item_condition_id column. We apply One hot encoding on item_condition_id column
  i/p= whole Dataframe
  o/p = OneHotEncoded csr matrix of item_condition_id column
  o/p shape = (?,5)
  """
  if train==True:
    item_condition = np.array(df['item_condition_id']).reshape(-1,1)
    item_condition_vec = OneHotEncoder()
    item_condition_vec.fit(item_condition)
    item_condition = csr_matrix(item_condition_vec.transform(item_condition))

    return item_condition,item_condition_vec
  else:
    item_condition = np.array(df['item_condition_id']).reshape(-1,1)
    item_condition = csr_matrix(vectorizer.transform(item_condition))
    return item_condition


#pre-process category_name

def pre_cat(df,main_cat_vec=None,subcat1_vec=None,subcat2_vec=None,training=True):
  """
  This function does the pre-processing of category_name variable. We first split this variable into 3 parts main category, sub category1 and subcategory 2
  One this new 3 columns One Hot encoding is appiled. Then all 3 a combined together and returend as output.
  df = Whole datfarme 
  main_cat_vec = Vectorizer for main category required during test time
  subcat1_vec =  Vectorizer for sub category1 required during test time
  subcat2_vec =  Vectorizer for sub category2 required during test time
  o/p = csr matrix of One hot encodeding of all 3 coloumn
  o/p shape = (?,973)
  """

  def pre_cat_name(x):
    """
    This function is used to pre-process the text.
    """
    x = str(x).lower()
    x = re.sub('[^A-Za-z0-9]+', ' ', x)
    x = re.sub(' +', ' ', x)
    x = x.replace(' ','_')
    return x

  def pre_item_condition_id(df,vectorizer=None,training=True):
    """
    This function returns the onehot encoded featuers.
    """
    if training==True:
      item_condition = df
      item_condition_vec = CountVectorizer(lowercase=False)
      item_condition_vec.fit(item_condition)
      item_condition = csr_matrix(item_condition_vec.transform(item_condition))

      return item_condition,item_condition_vec
    else:
      item_condition = df
      item_condition = csr_matrix(vectorizer.transform(item_condition))
      return item_condition

  train = df
  train["main_cat"] = train.category_name.str.extract("([^/]+)/[^/]+/[^/]+")
  train["subcat1"] = train.category_name.str.extract("[^/]+/([^/]+)/[^/]+")
  train["subcat2"] = train.category_name.str.extract("[^/]+/[^/]+/([^/]+)")
  train["main_cat"]  = train["main_cat"].apply(pre_cat_name)
  train["subcat1"]  = train["subcat1"].apply(pre_cat_name)
  train["subcat2"]  = train["subcat2"].apply(pre_cat_name)

  if training==True:
    main_cat, main_cat_vec  = pre_item_condition_id(train["main_cat"] )
    subcat1,subcat1_vec = pre_item_condition_id(train["subcat1"] )
    subcat2,subcat2_vec = pre_item_condition_id(train["subcat2"] )
    cat = hstack([main_cat,subcat1,subcat2])

    return cat,main_cat_vec,subcat1_vec,subcat2_vec
  else:
    main_cat  = pre_item_condition_id(train["main_cat"],main_cat_vec,training )
    subcat1 = pre_item_condition_id(train["subcat1"] ,subcat1_vec,training)
    subcat2 = pre_item_condition_id(train["subcat2"] ,subcat2_vec,training)
    cat = hstack([main_cat,subcat1,subcat2])

    return cat

def text_pre(data,tf_idf_vect=None,vec_type=None,training=True):

  """
  This function is to pre-process textual data and return TF-IDF vectorized output. 
  Here we combine name,barnd name and item description and convert it to single column.
  i/p = data with name,barnd name and item description  as input column
  o/p = TF-IDF vectorize output
  shape = (?,200000)
  """
  stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
          "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
          'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
          'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
          'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
          'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
          'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
          'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
          'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
          'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
          's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
          've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
          "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
          "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
          'won', "won't", 'wouldn', "wouldn't"])


  #ps = PorterStemmer() #create object of porterstemmer

  def decontracted(phrase):
    """
    This function is used for doing basic cleaning of sentences.
    i/p = string (sentence)
    o/p = string (clean sentence)
    """

    phrase = re.sub(r"won't", "will not", phrase) # expansion of word won't to will not
    phrase = re.sub(r"can\'t", "can not", phrase)

    phrase =  re.sub(r"http\S+", "", phrase)

    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = phrase.replace(':)', 'smiley').replace('(:', 'smiley').replace(':-)', 'smiley') # replace emojis with word smiley

    phrase = str(phrase).lower()# convert everything to lower
    phrase = re.sub('[^A-Za-z]+', ' ', phrase)# just keep alphabehets remove numbers and other unwanted chars
    phrase = re.sub(' +', ' ', phrase)# remove extra spaces

    phrase = ' '.join(e for e in phrase.split() if e not in stopwords)#remove stopwords
    #phrase = ' '.join(ps.stem(e) for e in phrase.split())#stemming of sentence

    return phrase

  df = data

  df['text'] = df['name'] + " " + df['brand_name'] + " " + df['item_description']

  df['text'] = df['text'].swifter.apply(decontracted)

  if training==True:
    if vec_type=='BOW':
      tf_idf_vect = CountVectorizer(ngram_range=(1,2), min_df=5,max_features=100000,dtype = np.float32)
    if vec_type=='tf-idf':
      tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=5,max_features=100000,dtype = np.float32)

    tf_idf_vect.fit(df['text'])
    final_tf_idf = tf_idf_vect.transform(df['text'])

    return final_tf_idf , tf_idf_vect
  else:
    final_tf_idf = tf_idf_vect.transform(df['text'])
  return final_tf_idf

def num_feature(df,vectorizer_nums=None,scale=None,training=True):
  """
  After some EDA and manually reviewing textual data I found out that there was lot of numerical information avaliable in text eg. 10ml , 2 packs, 10 lipsticks 512gb.
  This numerical data had impact on the price of item. So this a function does numerical vectorization of this data.So we first find patterns like (10ml,160gb 2 packs..etc)
  So now we have sentence "250ml 2 packs of xyz company" so here in "ml" column we had "250" and in packs column we add "2".At the end a sparse matrix is given out as output 
  """


  def get_featuers(phrase):
    """
    this function finds all possible numercial patterns in training data and accordingly updates the dictionary.
    """
    phrase = str(phrase).lower()
    phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase)
    phrase = re.sub(' +', ' ', phrase)

    nums = re.findall('[0-9]+.[A-Za-z]+',phrase)

    if len(nums)>0:
      for i in nums:
        a =  ''.join(re.findall('[A-Za-z]+',i))
        dic[a] = 0
    else:
      dic['no_data']=0

    return None

  def number(phrase): 
    """
    Finds all patterns and returns dictionary. eg {"ml":10,"gb":512},{"key(pattern)":"value(quantity)"}
    """
    send = dict()
    phrase = str(phrase).lower()
    phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase)
    phrase = re.sub(' +', ' ', phrase)

    nums = re.findall('[0-9]+.[A-Za-z]+',phrase)

    if len(nums)>0:
      for z in nums:
        a =  ''.join(re.findall('[A-Za-z]+',z))
        b =  ''.join(re.findall(r'\d+',z))
        b = int(b)
        send[a] = b
    else:
      send['no_data']=1

    return send

  def get_matrix(temp,dic):
    """
    This function is used to generate a csr matrix for numrecial vectorization
    """
    row_ind = []
    col_ind = []
    data = []
    row=0
    for i in temp:
      for z in i:
        if z in dic:
          col = dic[z]
          data_temp = i[z]
          row_ind.append(int(row))
          col_ind.append(int(col))
          data.append(int(data_temp))
      row+=1

    row_ind = np.array(row_ind,dtype=int)

    col_ind = np.array(col_ind,dtype=int)

    data = np.array(data,dtype=float)      

    matrix = csr_matrix((data, (row_ind, col_ind)), shape=(df.shape[0],len(dic)))

    return matrix


  if training==True:
    dic = dict()
    df['text'] = df['name'] + " " + df['item_description']
    df['text'].swifter.apply(get_featuers)
    c=0
    for i in dic:
      dic[i] = c
      c+=1

    temp = df['text'].apply(number)

    matrix = get_matrix(temp,dic)

    scaler = MaxAbsScaler(copy=False)
    scaler.fit(matrix)
    matrix = scaler.transform(matrix)

    return matrix,dic,scaler

  else:
    df['text'] = df['name'] + " " + df['item_description']
    temp = df['text'].swifter.apply(number)

    matrix = get_matrix(temp,vectorizer_nums)

    matrix = scale.transform(matrix)

    return matrix


#pre-process
def pre_process(df_train,df_test):
  #process shipping
  X_train_shipping = pre_shipping(df_train)
  X_test_shipping =  pre_shipping(df_test)

  #process item condition
  X_train_item_condition,item_condition_vec  = pre_item_condition_id(df_train)
  X_test_item_condition = pre_item_condition_id(df_test,item_condition_vec,train=False)

  del item_condition_vec

  #process cat
  X_train_cat,main_cat_vec,subcat1_vec,subcat2_vec = pre_cat(df_train)
  X_test_cat = pre_cat(df_test,main_cat_vec,subcat1_vec,subcat2_vec,training=False)

  del main_cat_vec,subcat1_vec,subcat2_vec

  #process text
  X_train_text,vectorizer_text = text_pre(df_train,vec_type='BOW')
  X_test_text = text_pre(df_test,vectorizer_text,training=False)
  del vectorizer_text


  #num featuers
  X_train_num,vectorizer_nums,scaler = num_feature(df_train)
  X_test_num = num_feature(df_test,vectorizer_nums,scaler,training=False)

  del vectorizer_nums,scaler

  #concat
  X_train_final = hstack([X_train_shipping,X_train_item_condition,X_train_cat,X_train_text,X_train_num]).tocsr().astype('float32')
  X_test_final = hstack([X_test_shipping,X_test_item_condition,X_test_cat,X_test_text,X_test_num]).tocsr().astype('float32')

  #scale
  # scaler = MaxAbsScaler(copy=False)
  # scaler.fit(X_train_final)

  # X_train_final = scaler.transform(X_train_final).tocsr().astype('float32')
  # X_test_final  = scaler.transform(X_test_final).astype('float32')

  return X_train_final,X_test_final


def mlp_1(X_train):
  tf.keras.backend.clear_session() 
  model_in = Input(shape=(X_train.shape[1],), dtype='float32', sparse=True)

  out = Dense(512, activation='tanh')(model_in)
  out = Dropout(0.25)(out)
  out = Dense(124, activation='tanh')(out)
  out = Dense(64, activation='tanh')(out)
  out = Dense(64, activation='tanh')(out)
  out = Dense(32, activation='tanh')(out)
  model_out = Dense(1)(out)
  model = Model(model_in, model_out)
  return model

def mlp_2(X_train):
  tf.keras.backend.clear_session() 
  model_in = Input(shape=(X_train.shape[1],), dtype='float32', sparse=True)
  out = Dense(512, activation='tanh')(model_in)
  out = Dropout(0.25)(out)
  out = Dense(64, activation='tanh')(out)
  out = Dense(1)(out)
  model = Model(model_in, out)
  return model


def mlp1_train(X_train,y_train,X_test):
  mlp1 = mlp_1(X_train)
  mlp1.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),loss='mean_squared_error',metrics='mean_squared_error')
  for i in range(5):
    mlp1.fit(x=X_train,y=y_train,batch_size=2**(5+i),epochs=1)

  y_pred_mlp1 = np.expm1(y_scaler.inverse_transform(mlp1.predict(X_test).reshape(-1, 1))[:, 0])

  return y_pred_mlp1

def mlp2_train(X_train,y_train,X_test):
  mlp2 = mlp_2(X_train)
  mlp2.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),loss='mean_squared_error',metrics='mean_squared_error')
  for i in range(5):
    mlp2.fit(x=X_train,y=y_train,batch_size=2**(5+i),epochs=1)

  y_pred_mlp2 = np.expm1(y_scaler.inverse_transform(mlp2.predict(X_test).reshape(-1, 1))[:, 0])
  return y_pred_mlp2


def lgbm_train(X_train,y_train,X_test):
  params ={'colsample_bytree': 0.75,
  'learning_rate': 0.72,
  'max_depth': 39,
  'min_child_weight': 37,
  'min_split_gain': 0.6,
  'num_leaves': 183,
  'reg_alpha': 0.2,
  'reg_lambda': 0.4,
  'subsample': 0.81,
  'n_estimators':800}
  lgbm = lgb.LGBMRegressor(n_jobs=1,early_stopping_rounds=None,**params)
  lgbm.fit(X_train,y_train)
  y_pred_lgbm = np.expm1(y_scaler.inverse_transform(lgbm.predict(X_test).reshape(-1, 1))[:, 0])
  return y_pred_lgbm

def sgr_train(X_train,y_train,X_test):
  sgr = SGDRegressor(max_iter=200000,n_iter_no_change=100,loss='squared_epsilon_insensitive',penalty='elasticnet',alpha=0.01)
  sgr.fit(X_train, y_train)
  y_pred_sgr = np.expm1(y_scaler.inverse_transform(sgr.predict(X_test).reshape(-1, 1))[:, 0])
  return y_pred_sgr



###################################################################################################################

func_extract_zip('/content/drive/My Drive/AAIC/CS/Data/train.tsv.7z','/content/')
func_extract_zip('/content/drive/My Drive/AAIC/CS/Data/test_stg2.tsv.zip','/content/')

df_train,df_test = func_load_data()
df_train = df_train[(df_train.price >= 3) & (df_train.price <= 2000)]

df_train = imputation(df_train)
df_test =  imputation(df_test)

print("1 done")
print('#'*50)
###################################################################################################################

# (2) Generate Data Encodings
y_scaler = StandardScaler(copy=False)
y_train = y_scaler.fit_transform(np.log1p(df_train['price'].values.reshape(-1, 1)))
ids = df_test.index.values.copy()
X_train,X_test = pre_process(df_train,df_test)
print("2 done")
print('#'*50)
del df_train,df_test
###################################################################################################################

preds_mlp1 = mlp1_train(X_train,y_train,X_test)
print("3 mlp1 done")
print('#'*50)
###################################################################################################################

preds_mlp2 = mlp2_train(X_train,y_train,X_test)
print("4 mlp2 done")
print('#'*50)
###################################################################################################################

# preds_lgbm = lgbm_train(X_train,y_train,X_test)
# print("5 lgbm done")
# print('#'*50)
###################################################################################################################

preds_sgr = sgr_train(X_train,y_train,X_test)
print("6 sgr done")
print('#'*50)
# ###################################################################################################################

pred_final = (preds_mlp1 + preds_mlp2 + preds_sgr)/4

# (7) Submissions output file
submission = pd.DataFrame({'test_id' : ids,'price' : pred_final})
submission.to_csv("submission.csv", index=False)
print('(7) done')


1 done
##################################################



2 done
##################################################
2894/2894 [==============================] - 48s 17ms/step - loss: 0.2362 - mean_squared_error: 0.2362
3 mlp1 done
##################################################
2894/2894 [==============================] - 47s 16ms/step - loss: 0.2124 - mean_squared_error: 0.2124
4 mlp2 done
##################################################
6 sgr done
##################################################
(7) done
